In [1]:
from langchain.chat_models.gigachat import GigaChat
from langchain.schema import HumanMessage, SystemMessage
import os
import vk_api
from vk_api.longpoll import VkLongPoll, VkEventType
from vk_api.utils import get_random_id

In [2]:
with open('gigachat_api_key.txt', 'r') as file:
    GIGACHAT_API_KEY = file.read().strip()
with open('vk_token.txt', 'r') as file:
    VK_TOKEN = file.read().strip()
token = VK_TOKEN
os.environ["GIGACHAT_CREDENTIALS"] = GIGACHAT_API_KEY
giga = GigaChat(profanity=False, temperature=0.6,  verify_ssl_certs=False)

In [3]:
system_message = SystemMessage(content=(
    f"""
Контекст: Вы - виртуальный ассистент, созданный для помощи в коммуникациях между населением Мирнинского Района Республики Саха (Якутия) и местной администрацией. Ваша задача - обрабатывать запросы от жителей, касающиеся различных аспектов жизни в регионе, включая муниципальные вопросы, социальные программы, инфраструктурные проекты и прочее. Вы должны предоставлять точную и актуальную информацию, соблюдая официальный и уважительный тон в общении.
Особенности региона: Якутия - крупнейший регион России с уникальными климатическими условиями и культурным наследием. Большая часть запросов может касаться особенностей жизни в суровых климатических условиях, развития социальной инфраструктуры и поддержки коренных народов Севера.
Типы запросов:
    Информация о муниципальных услугах (водоснабжение, отопление, дорожное строительство).
    Вопросы социальной поддержки (льготы, социальные выплаты).
    Информация о местных культурных и образовательных событиях.
    Вопросы, связанные с экологией и природопользованием.
    Обращения по вопросам местного самоуправления и участия в общественной жизни.
Тон общения: Официальный, уважительный, информативный. Важно демонстрировать понимание запросов и предоставлять исчерпывающие ответы, основанные на актуальной нормативно-правовой документации и официальной информации администрации.
Источники информации: Официальный сайт администрации Мирнинского Района Республики Саха (Якутия), муниципальные порталы, официальные публикации и пресс-релизы.
"""))

In [ ]:
class DialogueAgent:
    def __init__(self, name: str, system_message: SystemMessage, model: giga) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}:"
        self.reset()

    def reset(self):
        self.message_history = ["Вот разговор:"]

    def send(self) -> str:
        message = self.model([
            self.system_message,
            HumanMessage(content="\n".join(self.message_history + [self.prefix])),
        ])
        return message.content

    def receive(self, name: str, message: str) -> None:
        self.message_history.append(f"{name}: {message}")

    def get_response(self, user_input: str) -> str:
        self.receive(name="User", message=user_input)
        response = self.send()
        self.receive(name=self.name, message=response)
        return response

    def get_response_with_link(self, user_input: str) -> str:
        response = self.get_response(user_input)
        response += "\nДополнительная информация доступна на сайте администрации Мирнинского Района: https://www.xn----7sbab7amcgekn3b5j.xn--p1ai/administratsiya-mo/"
        
        return response

def send_message(vk, user_id, text):
    vk.method('messages.send', {'user_id': user_id, 'message': text, 'random_id': get_random_id()})

def handle_message(event, agent):
    vk = vk_api.VkApi(token=token)
    print('User input: ', event.text)
    response = agent.get_response_with_link(event.text)
    print('output: ', response)
    send_message(vk, event.user_id, response)

def main(token, agent):
    vk_session = vk_api.VkApi(token=token)
    longpoll = VkLongPoll(vk_session)
    for event in longpoll.listen():
        if event.type == VkEventType.MESSAGE_NEW and event.to_me:
            handle_message(event, agent)

name = "Виртуальный ассистент Сириус"
party = DialogueAgent(name=name, system_message=system_message, model=giga)
main(token, party)

User input:  Что можешь рассказать об администрации?
output:  Администрация Мирнинского района Республики Саха (Якутия) является исполнительным органом местного самоуправления, который осуществляет руководство и управление муниципальными вопросами на территории района. Она занимается решением различных вопросов, связанных с развитием района, обеспечением комфортной жизни его жителей и поддержанием социально-экономического развития.

Администрация района осуществляет широкий спектр деятельности, включая организацию муниципальных услуг, таких как водоснабжение, отопление, дорожное строительство, а также развитие социальной инфраструктуры, предоставление социальных выплат и льгот, организацию культурных и образовательных мероприятий, а также контроль за экологическими и природопользовательскими вопросами.

Официальный сайт администрации Мирнинского района Республики Саха (Якутия) является основным источником информации для жителей района. На сайте можно найти актуальные новости, официальн